In [93]:
import pandas as pd
import os

In [95]:
csv_files = [f for f in os.listdir('files/olds')]
for file in csv_files:
    df = pd.read_csv(f'files/olds/{file}')
    columns = ['Data', 'Hora UTC', 'PRECIPITA TOTAL, HORARIO (mm)' ]
    df = df[columns]
    df = df.drop(columns=['Hora UTC'])
    df['PRECIPITA TOTAL, HORARIO (mm)'] = df['PRECIPITA TOTAL, HORARIO (mm)'].str.replace(',', '.')
    df['PRECIPITA TOTAL, HORARIO (mm)'] = pd.to_numeric(df['PRECIPITA TOTAL, HORARIO (mm)'], errors='coerce')
    df[df['PRECIPITA TOTAL, HORARIO (mm)'] < 0] = None

    while df['PRECIPITA TOTAL, HORARIO (mm)'].isna().sum() > 0:
        df['PRECIPITA TOTAL, HORARIO (mm)'] = df['PRECIPITA TOTAL, HORARIO (mm)'].fillna(method='ffill').fillna(method='bfill')
    df_grouped = df.groupby("Data").sum().reset_index()
    file_name = file.replace('.CSV', '.csv')
    df_grouped.to_csv(f'files/{file_name}', index=False)
    


/tmp/ipykernel_935044/2463525049.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['PRECIPITA TOTAL, HORARIO (mm)'] = df['PRECIPITA TOTAL, HORARIO (mm)'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_935044/2463525049.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['PRECIPITA TOTAL, HORARIO (mm)'] = df['PRECIPITA TOTAL, HORARIO (mm)'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_935044/2463525049.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['PRECIPITA TOTAL, HORARIO (mm)'] = df['PRECIPITA TOTAL, HORARIO (mm)'].fillna(method='ffill').fillna(method='bfill')
/tmp/ipykernel_935044/2463525049.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in 

In [97]:
csv_files = [f for f in os.listdir('files') if os.path.isfile(os.path.join('files', f))]
for file in csv_files:
    df = pd.read_csv(f'files/{file}')
    df['Data'] = pd.to_datetime(df['Data'])
    df.to_csv(f'files/{file}', index=False)


In [104]:
combined_df = pd.concat([pd.read_csv(f'files/{file}') for file in csv_files])
combined_df.to_csv('files/combined_data.csv', index=False)

In [113]:
df = combined_df.copy()
df["Data"] = pd.to_datetime(df["Data"])
additional_data = {
    "Data": ["2011-03-27", "2011-03-28"],
    "PRECIPITA TOTAL, HORARIO (mm)": [0.0, 0.0]
}
additional_df = pd.DataFrame(additional_data)
additional_df["Data"] = pd.to_datetime(additional_df["Data"])
df = pd.concat([additional_df, df]).reset_index(drop=True)

df["Week"] = df["Data"].dt.to_period('W-SAT').apply(lambda r: r.start_time)

df_grouped = df.drop(columns=['Data']).groupby("Week").sum().reset_index()

df_grouped.head(10)

,Week,"PRECIPITA TOTAL, HORARIO (mm)"
0,2011-03-27,0.6
1,2011-04-03,22.4
2,2011-04-10,44.2
3,2011-04-17,11.2
4,2011-04-24,6.2
5,2011-05-01,7.2
6,2011-05-08,1.0
7,2011-05-15,15.0
8,2011-05-22,0.2
9,2011-05-29,0.6


In [114]:
df_grouped.to_csv('files/combined_data.csv', index=False)